In [32]:
import pandas as pd
import json

# Leer el archivo desde la ruta
archivo = "/Users/severinodonate/Downloads/planet_osm_rels_COMPLETO.csv"
df_rels = pd.read_csv(archivo, sep=",", quotechar='"')

# Convertir la columna 'tags' (en formato JSON) a diccionario Python
df_rels['tags_dict'] = df_rels['tags'].apply(lambda x: json.loads(x) if pd.notna(x) else {})

# Expandir los tags
tags_expandidos = pd.json_normalize(df_rels['tags_dict'])

# Unir al dataframe original sin perder 'tags'
df_rels_expandido = pd.concat([df_rels.drop(columns=['tags_dict']), tags_expandidos], axis=1)

# Guardar el archivo expandido
df_rels_expandido.to_csv("/Users/severinodonate/Downloads/planet_osm_rels_expandido.csv", index=False, sep=';', quoting=1)


In [33]:
# Número total de filas del dataset
total_filas = len(df_rels_expandido)

# Contar valores no nulos por columna
resumen = (
    df_rels_expandido.notna()
    .sum()
    .reset_index()
    .rename(columns={"index": "variable", 0: "valores"})
)

# Calcular el porcentaje de cobertura por variable
resumen["porcentaje"] = (resumen["valores"] / total_filas * 100).round(2)

# Ordenar de mayor a menor cobertura
resumen = resumen.sort_values(by="porcentaje", ascending=False).reset_index(drop=True)

# Guardar el resumen como CSV
resumen.to_csv("/Users/severinodonate/Downloads/resumen_variables_rels.csv", index=False, sep=';')
